<a href="https://colab.research.google.com/github/glrod666/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark


In [2]:
import os

# Find the latest version of spark 3.x from the Apache archive
spark_version = '3.4.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update -q
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop3.tgz
!tar xf spark-$SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder\
    .appName("Colab")\
    .getOrCreate()

# Print the Spark version to verify
print(spark.version)


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (128 kB/s)
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide i

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)




In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_data')
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import mean
from pyspark.sql.functions import round, col

four_bed_per_year = df.filter(df.bedrooms == 4)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("avg_price"))

four_bed_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-11-13| 358445.0|
|2019-05-08| 234979.5|
|2021-08-27|275547.25|
|2021-01-27|367520.57|
|2021-06-22| 297708.8|
|2021-10-11|283388.33|
|2020-08-24|301080.75|
|2022-03-28|338544.78|
|2019-06-04|306373.08|
|2021-12-18| 305264.3|
|2020-11-29|318153.89|
|2020-01-21| 420576.0|
|2020-07-24| 293661.5|
|2021-11-25|291762.27|
|2019-11-18|290259.83|
|2019-09-22| 284864.5|
|2020-08-05|243835.56|
|2019-11-01|254751.38|
|2021-10-02|303838.38|
|2020-08-29| 285457.3|
+----------+---------+
only showing top 20 rows



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_three_bath_per_year = df.filter((df.bedrooms == 3) & (df.bathrooms == 3)) \
    .groupBy("date") \
    .agg(round(mean("price"), 2).alias("avg_price"))
three_bed_three_bath_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-05-08|285754.71|
|2021-06-22| 306565.0|
|2021-01-27|385711.75|
|2020-08-24|319252.33|
|2021-11-13|348180.33|
|2022-03-28| 280464.4|
|2021-10-11| 289886.4|
|2021-12-18| 178508.5|
|2021-08-27|298209.33|
|2019-06-04| 223109.0|
|2020-08-05|304566.75|
|2019-11-18|319833.67|
|2020-01-21| 303045.8|
|2021-11-25|321816.67|
|2019-11-01| 279354.0|
|2020-07-24| 283740.0|
|2020-11-29| 366673.0|
|2021-10-02| 400307.5|
|2019-09-22| 350352.5|
|2020-04-30| 192065.5|
+----------+---------+
only showing top 20 rows



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_per_year = df.filter(
    (df.bedrooms == 3) &
    (df.bathrooms == 3) &
    (df.floors == 2) &
    (df.sqft_living >= 2000)
) \
    .groupBy("date") \
    .agg(round(mean("price"), 2).alias("avg_price"))
avg_price_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-06-22| 420422.0|
|2021-01-27| 446538.0|
|2021-12-18| 178508.5|
|2021-11-13| 697736.0|
|2021-08-27| 385686.0|
|2022-03-28| 204818.0|
|2021-10-11| 167892.0|
|2019-05-08| 391556.0|
|2019-11-01|201940.33|
|2019-11-18| 398256.0|
|2021-10-02| 402262.0|
|2019-09-22| 365867.0|
|2020-01-21| 293205.0|
|2020-04-30| 232217.0|
|2022-01-31| 284811.0|
|2022-05-19|302160.33|
|2020-08-29| 423501.0|
|2020-08-28| 296028.5|
|2019-11-21| 428167.0|
|2021-07-30| 346839.0|
+----------+---------+
only showing top 20 rows



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

import time

start_time = time.time()

avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_data
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY avg_price DESC
""")
avg_price_per_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.4888229370117188 seconds ---


In [9]:
# Create a temporary table 'home_sales'
df.createOrReplaceTempView('home_sales')

In [10]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_data")

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_view_uncached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY avg_price DESC
""")
avg_price_per_view_uncached.show()

uncached_runtime = time.time() - start_time
print(f"Uncached query runtime: {uncached_runtime} seconds")



+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Uncached query runtime: 3.30541729927063 seconds


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_sales', mode='overwrite')

In [14]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('p_sales')

In [15]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_sales')

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# Start time for the uncached query
start_time_uncached = time.time()

# Uncached query
avg_price_per_view_uncached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM p_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY avg_price DESC
""")
avg_price_per_view_uncached.show()

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows



In [17]:
# Calculate and print the runtime for the uncached query
uncached_runtime = time.time() - start_time_uncached
print("--- Uncached query runtime: %s seconds ---" % uncached_runtime)

--- Uncached query runtime: 1.1123387813568115 seconds ---


In [18]:
# Cache the table
spark.catalog.cacheTable("home_data")


In [19]:
# Cache the DataFrame
spark.sql("CACHE TABLE p_sales")


DataFrame[]

In [20]:
# Start time for the cached query
start_time_cached = time.time()

In [21]:
# Cached query
avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM p_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY avg_price DESC
""")
avg_price_per_view_cached.show()

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows



In [22]:
# Calculate and print the runtime for the cached query
cached_runtime = time.time() - start_time_cached
print("--- Cached query runtime: %s seconds ---" % cached_runtime)

--- Cached query runtime: 0.8616127967834473 seconds ---


In [23]:
# Compare the runtimes
print(f"Uncached runtime: {uncached_runtime} seconds")
print(f"Cached runtime: {cached_runtime} seconds")
print(f"Difference: {uncached_runtime - cached_runtime} seconds")


Uncached runtime: 1.1123387813568115 seconds
Cached runtime: 0.8616127967834473 seconds
Difference: 0.25072598457336426 seconds


In [24]:
# Output the result of the cached query for clarity
avg_price_per_view_cached.show()

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows



In [25]:
# Ensure uncaching of table after use
spark.sql("UNCACHE TABLE p_sales")

DataFrame[]

In [26]:
# Check if the table is cached
print(spark.catalog.isCached('home_data'))

True


In [27]:
# Run the query again with caching and measure the runtime

import time
start_time = time.time()

avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_data
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY avg_price DESC
""")
avg_price_per_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 0.5711574554443359 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_data")

In [29]:
# 15. Check if the home_sales is no longer cached
spark.stop()
